<a href="https://colab.research.google.com/github/Shreya2803/Bengali-Language-Model/blob/main/scrape_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import requests
!pip install requests_html
!pip install google
!pip install selenium
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from googlesearch import search
#import pdfplumber
from IPython.display import display, HTML, Javascript
import ipywidgets as widgets
from selenium import webdriver
import webbrowser
from IPython.display import clear_output
import os, pickle
import json
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.6 MB/s 
     |████████████████████████████████| 140 kB 38.2 MB/s 
     |████████████████████████████████| 106 kB 53.2 MB/s 
     |████████████████████████████████| 127 kB 51.9 MB/s 
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24590 sha256=b58d188136e7701a81b735d556df2d97a7b84af4653c0654d7fa71f3d4ea2164
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 5.4 MB/s 
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
     |████████████████████████████████| 384 kB 56.6 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


/usr/local/lib/python3.7/dist-packages/trio/_core/_multierror.py:411: RuntimeWarning: IPython detected, but you already have a custom exception handler installed. I'll skip installing Trio's custom handler, but this means exception groups will not show full tracebacks.
  category=RuntimeWarning,


In [3]:
class ScrapeData():
    """
    Python tool to scrap text data from different languages. 3 options available:
    
    - scrape from link: provided with a link, the tool scrapes through the text body from the webpage 
      corresponding to the link.
      
    - scrape from keyword: provided with a keyword, the tool performs a google search and retrieves text from 
      a priority domain webpage(eg: wikipedia) or top google search result.
      
    - scrape from document: provided with a pdf document, the tools accesses the text using pdfplumber 
      python package. Note that some portion of the text may not be returned properly.
    """
    
    def __init__(self):
        """
        Initialise parameters for google search and priority website 
        """
        
        self.parser = 'html.parser'
        self.tld = 'co.in'
        self.search_num = 10
        self.search_stop = 10
        self.pause = 2
        self.search_priority = 'wikipedia'
        self.auto_return_index = 0
        
    def read_from_link(self, link, replace_list=['\n']):
        """
        This function accesses the text content from a webpage link using beautiful soup. To clean the text, 
        provide the list of charecters to be removed in replace_list.
        This may not work for js webpages, cloudfare ddos protected pages etc
        """
        #use header if access is denied.
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}

        page = requests.get(link, headers=headers)
        page = requests.get(link)
        soup = BeautifulSoup(page.content, self.parser)
        data = []
        for i in range(len(soup.find_all('p'))):
            text = soup.find_all('p')[i].get_text()
            for j in range(len(replace_list)):
                text = text.replace(replace_list[j],'')
            if len(text)>0:
                data.append(text)
        return data

    def google_search(self, search_keyword, priority=None):
        """
        This function performs google search on the input keyword. Priority can be provided to a particular 
        website (Eg: wikipedia) 
        """
        search_links = []
        for link in search(search_keyword, 
                           tld=self.tld, 
                           num=self.search_num, 
                           stop=self.search_stop, 
                           pause=self.pause):
            search_links.append(link)
            
            if priority is not None:
                if self.search_priority in link:
                    return link
        return search_links[self.auto_return_index]
                
    
    def read_from_doc(self, document):
        """
        This function extracts text from pdf using pdfplumber tool.
        """
        data = []
        with pdfplumber.open(document) as pdf:
            for i in range(len(pdf.pages)):
                page = pdf.pages[i]
                text = page.extract_text()
                if text is not None:
                    text = text.replace('\n', ' ')
                    data.append(text) 
        return data
    
    def read_page(self, search_keyword=None, link=None, document=None):
        if search_keyword == link == document == None:
            raise Exception('Provide link, keyword or document to scrape from')
            
        if search_keyword is not None:
            keyword_link = self.google_search(search_keyword, priority=self.search_priority)
            text = self.read_from_link(keyword_link)
            return(text)
            
        if link:
            text = self.read_from_link(link)
            return(text)
                
        if document:
            text = self.read_from_doc(document)
            print(f'{len(text)} pages found')
            return(text)
            
scrape_tool = ScrapeData()

<h4>Read from english website</h4>

In [ ]:
scrape_tool.read_page(link='https://en.wikipedia.org/wiki/Agriculture')

['Agriculture or farming is the practice of cultivating plants and livestock.[1] Agriculture was the key development in the rise of sedentary human civilization, whereby farming of domesticated species created food surpluses that enabled people to live in cities. The history of agriculture began thousands of years ago. After gathering wild grains beginning at least 105,000 years ago, nascent farmers began to plant them around 11,500 years ago. Pigs, sheep, and cattle were domesticated over 10,000 years ago. Plants were independently cultivated in at least 11 regions of the world. Industrial agriculture based on large-scale monoculture in the twentieth century came to dominate agricultural output, though about 2 billion people still depended on subsistence agriculture.',
 "The major agricultural products can be broadly grouped into foods, fibers, fuels, and raw materials (such as rubber). Food classes include cereals (grains), vegetables, fruits, oils, meat, milk, eggs, and fungi. Over 

<h4>Read from english keyword</h4>

In [ ]:
scrape_tool.read_page(search_keyword='agriculture')

<h4>Read from english pdf document</h4>

In [ ]:
scrape_tool.read_page(document='document_name.pdf')

---

<h3>Read from list of links with GUI</h3>

In [ ]:
class ActiveScrappingGUI():
    def __init__(self, links):
        if not isinstance(links, list):
            raise Exception('link input shoudl be present as a python list')
        self.linkIdx = 0
        self.links = links
        self.approvedList = {}
        self.storedText = {}
        self.defaultColor  = '#EEEEEE'
        self.approvedColor = 'lightgreen'
        self.rejectedColor = '#FF4500'
        self.nextButton = widgets.Button(description = 'Next')
        self.approve = widgets.Button(description = 'Approve', disabled=True)
        self.reject = widgets.Button(description = 'Reject', disabled=True)
        self.nextButton.add_class("red_label")
        self.approve.add_class("red_label")
        self.reject.add_class("red_label")
        self.output = widgets.Output()
        display(widgets.HBox((self.nextButton, self.approve,self.reject )), self.output, 
        HTML("<style>.red_label { font-weight: bold}</style>"),
        HTML("<style>.red_label { font-family:calibri}</style>"),
        HTML("<style>.red_label { font-size:16px}</style>"))
        
    def on_button_clicked_approve(self, b):
        with self.output:
            self.nextButton.disabled = False
            self.approve.style.button_color = self.approvedColor
            self.reject.style.button_color = self.defaultColor
            self.approvedList[self.links[self.linkIdx-1]] = True
                
    def on_button_clicked_reject(self, b):
        with self.output:
            self.nextButton.disabled = False
            self.approve.style.button_color = self.defaultColor
            self.reject.style.button_color = self.rejectedColor
            self.approvedList[self.links[self.linkIdx-1]] = False
                
    def on_button_clicked(self, b):
        with self.output:
            if  self.linkIdx == len(self.links):
                clear_output()
                print('All links visited. Approved link can be accessed with "gui.approvedList"')
                self.reject.disabled = True
                self.approve.disabled = True
                self.nextButton.disabled = True
                self.reject.style.button_color = self.defaultColor
                self.approve.style.button_color = self.defaultColor
            else:
                clear_output()
                self.reject.disabled = True
                self.approve.disabled = True
                self.nextButton.disabled = True
                self.reject.style.button_color = self.defaultColor
                self.approve.style.button_color = self.defaultColor
                print('Extracting..')
                text = scrape_tool.read_page(link=self.links[self.linkIdx])
                clear_output()
                self.storedText[self.links[self.linkIdx]] = text
                for para in text:
                    print(para, '\n')
                webbrowser.open(self.links[self.linkIdx])
                self.reject.disabled = False
                self.approve.disabled = False
                self.linkIdx += 1
            
    def start(self):
        self.approve.on_click(self.on_button_clicked_approve)
        self.reject.on_click(self.on_button_clicked_reject)
        self.nextButton.on_click(self.on_button_clicked)
    
    def save(self, path, topic_name):
        existingFiles = set(os.listdir(path))
        for key, item in self.approvedList.items():
            if item == True:                
                    name = os.path.join(path, '{topic_name}_'+str(time.time())+'.json')
                    if name in existingFiles:
                        raise Exception('A file already exists in the save path for the current file. '\
                                        'Change saving procedure for your setup')
                    with open(name, 'w') as f: 
                        json.dump({'link':key,
                                  'text':self.storedText[key]}, f)

In [ ]:
links = set(['https://en.wikipedia.org/wiki/Agriculture', 'https://www.bbc.com/hindi/india-56901831',
        'https://en.wikipedia.org/wiki/Main_Page'])
topic_name = []
# existingFiles = set(os.listdir(path))
# existing_links = set()
# for filename in existingFiles:
#     with open(os.path.join(path, filename), "rb") as read_file:
#         data = json.load(read_file)
#     link = data['link']
#     existing_links.add(link)
# existing_links  
# for link in links:
#     if link in existing_links:
#         links.pop(link)
#load links from csv into a list
gui = ActiveScrappingGUI(links)
gui.start()

Output()

In [ ]:
gui.approvedList

{'https://en.wikipedia.org/wiki/Agriculture': True,
 'https://www.bbc.com/hindi/india-56901831': False,
 'https://en.wikipedia.org/wiki/Main_Page': True}

In [ ]:
#store this as .pickle if rejected links are processed at a later time.
linkStatusPath = 'status.pickle'
with open(linkStatusPath, 'wb') as handle:
    pickle.dump(gui.approvedList, handle, protocol=pickle.HIGHEST_PROTOCOL)

<b> Save approved text </b>

In [ ]:
#provide your folder path
# gui.save(path='saved')

In [ ]:
# f = json.load('saved/extracted_file_1.json')

---

<b>Example on how to proceed with rejected links (Work In Progress)<b>

In [ ]:
class HandleRejects():
    def __init__(self, storedGUIStatusDict, openWebPage = True, ):
        self.approve = widgets.Button(description = 'Approve Changes')
        self.refresh = widgets.Button(description = 'Refresh Changes')
        self.discard = widgets.Button(description = 'Unable to process')
        self.output = widgets.Output()
        self.openWebPage = openWebPage
        self.defaultColor  = '#EEEEEE'
        self.approve.disabled = True
        self.discard.disabled = True
        self.approve.add_class("layout")
        self.refresh.add_class("layout")
        self.discard.add_class("layout")
        with open(linkStatusPath, 'rb') as handle:
            self.statusDct = pickle.load(handle)
        self.resolvedLinks = {}
        self.discardedLinks = {}
        self.initalVal = None
        self.initalVal = self.reportStatus()
        self.linkIdx = 0
        self.visitSet = set()
        self.linkFromStatus = [key for key, item in self.statusDct.items() if item == False]
        
    def reportStatus(self):
        if self.initalVal == None:
            toBeResolved = sum([1 if item == False else 0 for key, item in self.statusDct.items()])
            return toBeResolved
        else:
            self.remainingCount = self.initalVal - len(self.resolvedLinks) - len(self.discardedLinks)
            print(f'{self.remainingCount}/{self.initalVal} links left to be resolved')
    
    def on_button_clicked_approve(self, b):
        with self.output:
            self.approve.style.button_color = self.defaultColor
            self.resolvedLinks[self.currentLink] = self.currentText
            self.linkIdx += 1 
            clear_output()
            print(f'Text extracted from {self.currentLink} approved')
            self.reportStatus()
            self.discard.disabled = True
            self.approve.disabled = True
    
    def on_button_clicked_discard(self, b):
        with self.output:
            self.discard.style.button_color = self.defaultColor
            self.discardedLinks[self.currentLink] = self.currentText
            self.linkIdx += 1 
            clear_output()
            print(f'Text extracted from {self.currentLink} needs manual inspection. Unable to proceed with web scraping code')
            self.reportStatus()
            self.approve.disabled = True
            self.discard.disabled = True
            
    def on_button_clicked_refresh(self, b):
        with self.output:
            if len(self.linkFromStatus) == 0:
                raise Exception('No sentences dound in dict')
            self.approve.disabled = False
            self.discard.disabled = False
            self.reportStatus()
            print('Extracting..')
            if self.linkIdx >= len(self.linkFromStatus):
                clear_output()
                self.discard.disabled = True
                self.approve.disabled = True
                self.refresh.disabled = True
                raise Exception('All links visited. : )')
            self.currentLink = self.linkFromStatus[self.linkIdx]
            if self.currentLink not in self.resolvedLinks.keys():
                if self.currentLink not in self.visitSet:
                    self.visitSet.add(self.currentLink)
#                     webbrowser.open(self.currentLink)
                    print(self.currentLink)
                self.currentText = self.readFn(self.currentLink)
                clear_output()
                for line in self.currentText:
                    print(line, '\n')
    
    def start(self, readFn):
        display(widgets.HBox((self.approve, self.refresh, self.discard)), self.output, 
        HTML("<style>.layout { font-weight: bold}</style>"),
        HTML("<style>.layout { font-family:calibri}</style>"),
        HTML("<style>.layout { font-size:16px}</style>"))
        self.readFn = readFn
        self.refresh.on_click(self.on_button_clicked_refresh)
        self.approve.on_click(self.on_button_clicked_approve)
        self.discard.on_click(self.on_button_clicked_discard)
    
    def save(self, path):
        existingFiles = set(os.listdir(path))
        for key, item in self.resolvedLinks.items():
            if item == True:                
                    name = os.path.join(path, 'extracted_file_'+str(time.time())+'.json')
                    if name in existingFiles:
                        raise Exception('A file already exists in the save path for the current file. '\
                                        'Change saving procedure for your setup')
                    with open(name, 'w') as f: 
                        json.dump({'link':key,
                                  'text':self.storedText[key]}, f)
   
    def saveDiscarded(self, path):
        with open(path, 'w') as f:
            data = list(self.discardedLinks.keys())
            for line in data:
                f.write(line+'\n')
                
linkStatusPath = 'status.pickle'
handle = HandleRejects(storedGUIStatusDict=linkStatusPath);

FileNotFoundError: ignored

In [ ]:
def read_from_link_custom(link, replace_list=['\n']):  
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    data = []
    for i in range(len(soup.find_all('p'))):
        text = soup.find_all('p')[i].get_text()
        for j in range(len(replace_list)):
            text = text.replace(replace_list[j],'')
        if len(text)>0:
            data.append(text)
    return data  

handle.start(readFn=read_from_link_custom)

Output()

In [ ]:
#resolved links with extracted text present here
handle.resolvedLinks.keys()
#links which need manual inspection present here
# list(handle.discardedLinks.keys())

dict_keys(['https://en.wikipedia.org/wiki/Agriculture'])

In [ ]:
handle.save('saveResolved')
handle.saveDiscarded('furtherInspection/day1links.txt')

---

<b>Read from hindi webpage</b>

In [ ]:
# scrape_tool.read_page(link="https://hi.wikipedia.org/wiki/%E0%A4%95%E0%A5%83%E0%A4%B7%E0%A4%BF")
scrape_tool.read_page(link="https://www.bbc.com/hindi/india-56901831")

<b>Read from hindi document</b>

In [ ]:
scrape_tool.read_page(document='RedRidingHood-H-2mb.pdf')

<b>Read from kannada webpage</b>

In [ ]:
# scrape_tool.read_page(link="https://kn.wikipedia.org/wiki/%E0%B2%B8%E0%B2%BE%E0%B2%B5%E0%B2%AF%E0%B2%B5_%E0%B2%AC%E0%B3%87%E0%B2%B8%E0%B2%BE%E0%B2%AF")
scrape_tool.read_page(link="https://kannada.asianetnews.com/karnataka-districts/bjp-mla-g-somashekara-reddy-talks-lockdown-in-karnataka-grg-qs9n0r")

In [3]:
class ScrapeData():
    """
    Python tool to scrap text data from different languages. 3 options available:
    
    - scrape from link: provided with a link, the tool scrapes through the text body from the webpage 
      corresponding to the link.
      
    - scrape from keyword: provided with a keyword, the tool performs a google search and retrieves text from 
      a priority domain webpage(eg: wikipedia) or top google search result.
      
    - scrape from document: provided with a pdf document, the tools accesses the text using pdfplumber 
      python package. Note that some portion of the text may not be returned properly.
    """
    
    def __init__(self):
        """
        Initialise parameters for google search and priority website 
        """
        
        self.parser = 'html.parser'
        self.tld = 'co.in'
        self.search_num = 10
        self.search_stop = 10
        self.pause = 2
        self.search_priority = 'wikipedia'
        self.auto_return_index = 0
        
    def read_from_link(self, link, replace_list=['\n']):
        """
        This function accesses the text content from a webpage link using beautiful soup. To clean the text, 
        provide the list of charecters to be removed in replace_list.
        This may not work for js webpages, cloudfare ddos protected pages etc
        """
        #use header if access is denied.
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}

        page = requests.get(link, headers=headers)
        page = requests.get(link)
        soup = BeautifulSoup(page.content, self.parser)
        data = []
        # for i in range(len(soup.find_all("div", {"id":"div993.1.3.2.2541663"}))):
        #     text = soup.find_all("div", {"id":"div993.1.3.2.2541663"})[i].get_text()
        # for i in range(len(soup.find_all("div", {"class": "description"}))):
        #     text = soup.find_all("div", {"class": "description"})[i].get_text()
        # for i in range(len(soup.find_all("div", {"style": "text-align: justify;"}))):
        #     text = soup.find_all("div", {"style": "text-align: justify;"})[i].get_text()
        # for i in range(len(soup.find_all(['h2','h3','p','li']))):
        #     text = soup.find_all(['h2','h3','p','li'])[i].get_text() 
        # for i in range(len(soup.find_all('span'))):
        #     text = soup.find_all('span')[i].get_text()
        for i in range(len(soup.find_all('p'))):
            text = soup.find_all('p')[i].get_text()
            for j in range(len(replace_list)):
                text = text.replace(replace_list[j],'')
                text = text.replace('\t', '')
                text = text.replace('\xa0', '')
                text = text.replace('\r', '')
                text = text.replace('\u200c', '')      
                text = text.replace('\u200b\u200b\u200b\u200b\u200b\u200b\u200b', '')  
                text = text.replace('&apos', '')
                text = text.replace('\uf0b4', '')
                
            if len(text)>0:
                data.append(text)
        return data
    

    def google_search(self, search_keyword, priority=None):
        """
        This function performs google search on the input keyword. Priority can be provided to a particular 
        website (Eg: wikipedia) 
        """
        search_links = []
        for link in search(search_keyword, 
                           tld=self.tld, 
                           num=self.search_num, 
                           stop=self.search_stop, 
                           pause=self.pause):
            search_links.append(link)
            
            if priority is not None:
                if self.search_priority in link:
                    return link
        return search_links[self.auto_return_index]
                
    
    def read_from_doc(self, document):
        """
        This function extracts text from pdf using pdfplumber tool.
        """
        data = []
        with pdfplumber.open(document) as pdf:
            for i in range(len(pdf.pages)):
                page = pdf.pages[i]
                text = page.extract_text()
                if text is not None:
                    text = text.replace('\n', ' ')
                    data.append(text) 
        return data
    
    def read_page(self, search_keyword=None, link=None, document=None):
        if search_keyword == link == document == None:
            raise Exception('Provide link, keyword or document to scrape from')
            
        if search_keyword is not None:
            keyword_link = self.google_search(search_keyword, priority=self.search_priority)
            text = self.read_from_link(keyword_link)
            return(text)
            
        if link:
            text = self.read_from_link(link)
            return(text)
                
        if document:
            text = self.read_from_doc(document)
            print(f'{len(text)} pages found')
            return(text)
            
scrape_tool = ScrapeData()

**Create a *txt* file to store all data of each domain**

In [67]:
import json
import string
file = '/content/drive/MyDrive/web_scraper/General/Transport_30.json'
with open('/content/drive/MyDrive/web_scraper/General/Transport_30.json', "rb") as read_file:
  data = json.load(read_file)
  #deletes the first key (which is a link) from the json data 
  del data['link']
  vals = list( data.values() )
  keys = list( data.keys() )
  count = 0
  # create an empty string
  text=""
  for val in vals:
    if len(val)>1:
      for part in val:
        text = text +" " +(part)+"\n"
    else:
      text = text + val[0]+"\n" 
      count += 1
    #deletes punctuation(Ex : , ! ?)
    text=text.translate(text.maketrans('','',string.punctuation))
    print(text)
with open("/content/drive/MyDrive/web_scraper/Word Frequency/Raw Data/General Raw Data.txt", "a") as write_file:
   write_file.write(text)


 বিকেল হতেই ধর্মতলার বাস টার্মিনাসে দেখা গেল শ’য়ে শ’য়ে অন্য জেলার যাত্রীদের ভিড়। 
 হাতে আর মাত্র কয়েক ঘণ্টা। তার মধ্যেই ফিরে যেতে হবে বাড়ি। পরিবারপরিজনের কাছে। না হলেই আটকে পড়তে হবে। রবিবার থেকে রাজ্যে কার্যত লকডাউনের ঘোষণা হওয়ার পর থেকেই ভিড় জমতে শুরু করেছিল শহরের বাস টার্মিনাসগুলোতে। যাঁরা ভিন্জেলা থেকে কর্মসূত্রে কলকাতা শহরে এসেছিলেন শনিবার বিকেল থেকেই তাঁদের মধ্যে বাড়ি ফেরার তাড়াহুড়ো লক্ষ করা গেল।
 বিকেল হতেই ধর্মতলার বাস টার্মিনাসে দেখা গেল শ’য়ে শ’য়ে অন্য জেলার যাত্রীদের ভিড়। তাঁদের হাতে বোঁচকা ব্যাগ লটবহর। অনেকের মুখেই মাস্ক নেই। কারও মুখে মাস্কের পরিবর্তে গামছা জড়ানো। কোভিড পরিস্থিতির মধ্যেও তাঁদের ঠাসাঠাসি করেই উঠতে হচ্ছে বাসে। দূরত্ব বিধির তোয়াক্কা না করে কোনও রকমে চেপেচুপে বাসের মাথাতে জায়গা করে নিচ্ছেন তাঁরা।
 কোভিড সংক্রমণে রাশ টানতে রবিবার থেকে রাজ্যে কার্যত লকডাউন ঘোষণা করেছে সরকার। জারি থাকবে আগামী ১৫ দিন পর্যন্ত। সরকারের তরফে স্পষ্ট জানিয়ে দেওয়া হয়েছে জরুরি পরিষেবা বাদে বন্ধ থাকবে সমস্ত সরকারি এবং বেসরকারি অফিস দফতর। আগেই রাজ্যে লোকাল ট্রেন বন্ধ করার সিদ্ধান্ত নি

**Remove characters (|, ' ,>>)from the text file and save it in a different file**

In [93]:
with open('/content/drive/MyDrive/web_scraper/Word Frequency/Raw Data/General Raw Data.txt', 'r') as read_file, open('/content/drive/MyDrive/web_scraper/Word Frequency/Raw Data Final/General Raw Data Final.txt', 'w') as outfile:
    data = read_file.read()
    remove_char='-♦©।‘‘‧—…‘’“”～®→•●'
    translation_table = str.maketrans('', '', remove_char)
    data = data.translate(translation_table)
    # temp = re.sub('[-♦©।‘‘‧’র—…’s‘’»ো“”～®→•]', ' ', temp)
    # temp = infile.read().replace("-", " ")
    outfile.write(data)

 **Count the total number of Lines  from the extracted Json files**

In [94]:
file = '/content/drive/MyDrive/web_scraper/Word Frequency/Raw Data Final/General Raw Data Final.txt'
with open('/content/drive/MyDrive/web_scraper/Word Frequency/Raw Data Final/General Raw Data Final.txt', "rb") as read_file:
    data =len(read_file.readlines())
    print('Total Number of lines:', data)
with open("/content/drive/MyDrive/web_scraper/Word Frequency/Frequency/General Word Frequency.txt", "w") as write_file:
  write_file.write("Total number of lines: "+str(data) +"\n")


Total Number of lines: 34963


**Word Frequency**

In [95]:
import operator
text = open("/content/drive/MyDrive/web_scraper/Word Frequency/Raw Data Final/General Raw Data Final.txt", "r")

# Create an empty dictionary
d = dict()

# Loop through each line of the file
for line in text:
	# Remove the leading spaces and newline character
	line = line.strip()

	# Convert the characters in line to
	# lowercase to avoid case mismatch
	line = line.lower()

	# Split the line into words
	words = line.split(" ")
						

	# Iterate over each word in line
	for word in words:
		# Check if the word is already in dictionary
		if word in d:
			# Increment count of word by 1
			d[word] = d[word] + 1
		else:
			# Add the word to dictionary with count 1
			d[word] = 1
sorted_d = dict( sorted(d.items(), key=operator.itemgetter(1),reverse=True))
with open("/content/drive/MyDrive/web_scraper/Word Frequency/Frequency/General Word Frequency.txt", "a") as write_file:
  # Print the contents of dictionary
  for key in list(sorted_d.keys()):
    write_file.write(key+ ":"+str(sorted_d[key]) +"\n")
# 	print(key, ":", d[key])


**Converts the list into dictionary where links are saved as key and content is saved as value and then the file is saved in Directory**

In [146]:
import json
link="https://bn.how-what-finance.com/12738748-how-to-choose-a-credit-counseling-agency-14-steps-with-pictures"
content = scrape_tool.read_page(link="https://bn.how-what-finance.com/12738748-how-to-choose-a-credit-counseling-agency-14-steps-with-pictures")
#delete words from the extracted data from the last
# del content[10:]
# del content[-10:]
# del content[:2]
# content.remove("Ready to Invest?Talk to our investment specialistDisclaimer")
# content.remove("প্রতীকী ছবি।   ")
# content.remove(" ")
# content.remove("জমি বা শস্যগুলিতে জলের জলের সরবরাহ সাধারণত বৃদ্ধির পক্ষে সহায়তা করে চ্যানেলগুলির মাধ্যমে।জল বা medicationষধের অবিচ্ছিন্ন প্রবাহের সাথে কোনও অঙ্গ বা ক্ষত ধুয়ে ফেলার প্রক্রিয়া।গর্ত ইত্যাদি দিয়ে শুকনো জমি সরবরাহ করা(medicineষধ) জলে বা medicষধিযুক্ত দ্রবণ দিয়ে ফ্লাশ করে বা ধৌত করে ক্ষত বা দেহের অঙ্গ পরিষ্কার করে")
# # del content[11:]
# Data to be written
dictionary ={
    "link" :link,
    "content" :content
}
json_object = json.dumps(dictionary, ensure_ascii=False,indent = 4) 
print(json_object)    
with open("/content/drive/MyDrive/web_scraper/Finance/Credit counseling_2.json", "w") as json_file:
    json.dump(dictionary, json_file ,ensure_ascii=False)

{
    "link": "https://bn.how-what-finance.com/12738748-how-to-choose-a-credit-counseling-agency-14-steps-with-pictures",
    "content": [
        "You don't have permission to access /12738748-how-to-choose-a-credit-counseling-agency-14-steps-with-pictureson this server."
    ]
}


**Read from bengali webpage**

In [ ]:
# scrape_tool.read_page(link="https://www.anandabazar.com/rabibashoriyo/%E0%A6%97-%E0%A6%9C-%E0%A6%9A%E0%A6%B0%E0%A6%B8-%E0%A6%B9-%E0%A6%B6-%E0%A6%B6-1.313437")
scrape_tool.read_page(link="https://bn.wikipedia.org/wiki/%E0%A6%B8%E0%A7%8D%E0%A6%AC%E0%A6%BE%E0%A6%B8%E0%A7%8D%E0%A6%A5%E0%A7%8D%E0%A6%AF%E0%A7%87%E0%A6%B0_%E0%A6%89%E0%A6%AA%E0%A6%B0_%E0%A6%A4%E0%A6%BE%E0%A6%AE%E0%A6%BE%E0%A6%95%E0%A7%87%E0%A6%B0_%E0%A6%AA%E0%A7%8D%E0%A6%B0%E0%A6")

[]

**Read the saved Json File**

In [ ]:
import json

file = '/content/drive/MyDrive/web_scraper/Agriculture/Farmer's_bill_india_(new)_1.json'
with open('/content/drive/MyDrive/web_scraper/Agriculture/Farmer's_bill_india_(new)_1.json', "rb") as read_file:
    data = json.load(read_file)
data

{'https://bengali.krishijagran.com/education': ['নবোদয় বিদ্যালয়ে ভর্তি হতে চান, তাহলে এইভাবে আবেদন করুন',
  'দেশের সকল অভিভাবকদের জন্য সুখবর রয়েছে যারা তাদের সন্তানদের নবোদয় বিদ্যালয়ে পাঠাতে চান। প্রকৃতপক্ষে, নবোদয় বিদ্যালয় সমিতি',
  ' ‘৮৯ হাজার সংখ্যাটা নেহাত কম নয়’,চাকরি প্রার্থীদের আশ্বস্ত করলেন মমতা ',
  'নিয়োগ দুর্নীতি নিয়ে গোটা বাংলা যখন তোলপার ঠিক তখনই শিক্ষাক্ষেত্রে বিপুল নিয়োগের কথা জানালেন মুখ্যমন্ত্রী মমতা বন্দ্যোপাধ্যায়।',
  'ভারতীয় সেনাবাহিনীতে প্রচুর নিয়োগ ,জারি হল বিজ্ঞপ্তি ',
  'ভারতীয় সেনাবাহিনীতে এনসিসি স্পেশাল এন্ট্রি স্কিমের আওতায় নিয়োগের বিজ্ঞপ্তি জারি হয়েছে। উপযুক্ত শিক্ষাগত যোগ্যতা থাকলে আগ্রহী প্রার্থীরা এই',
  'IBPS RRB Admit Card 2022: আইবিপিএস এর অ্যাডমিট কার্ড ডাউনলোড করুন,জানুন পদ্ধতি',
  'মোট আট হাজারেরও বেশি শূন্য পদে যোগ্য প্রার্থী বাছাই করা হবে। প্রাথমিক ও মূল পরীক্ষার ভিত্তিতে প্রার্থী বাছাই করা হবে। কোনো ইন্টারভিউ নেওয়া হবে না।',
  'প্রাইমারি দুর্নীতি নিয়ে সিবিআই কে সিট গঠনের নির্দেশ দিল কলকাতা হাইকোর্ট ',
  'প্রাইমারিতে নিয়োগ দুর্নীতি